In [24]:
# Install library yang diperlukan
import sys
!{sys.executable} -m pip install transformers
!{sys.executable} -m pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


c:\Python311\python.exe: No module named pip3


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import LabelEncoder
from torch import nn
import pandas as pd
import re

c:\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load Dataset
dataset_path = "IKN_Nusantara.csv"  
output_path = "sentiment_analysis_results.csv"
df = pd.read_csv(dataset_path)
# Pilih kolom tweet yang akan digunakan
tweets = df["title"].tolist()

print(df["title"])

0       Presiden Joko Widodo menggelar rapat dengan se...
1       Halo #SobATRBPN, simak bersama yuk! Kegiatan W...
2       Reposted from @atrbpn - Halo #SobATRBPN, Kegia...
3       Halo #SobATRBPN, simak bersama yuk! Kegiatan W...
4       #repost @atr_bpn --- Halo #SobATRBPN, simak be...
                              ...                        
1222    Menteri Investasi Bahlil Lahadalia mengatakan,...
1223    Minimnya minat investor swasta dalam proses pe...
1224    RT @kompascom: Kepala Otorita Ibu Kota Nusanta...
1225    Kepala Otorita Ibu Kota Nusantara (OIKN) Bamba...
1226    Kemenkeu menggelontorkan sederet insentif perp...
Name: title, Length: 1227, dtype: object


In [6]:
# Preprocessing function to remove specific symbols
def preprocess_text(text):
    # Remove specific symbols using regex
    processed_text = re.sub(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", "", text)
    return processed_text

# Apply preprocessing to the 'description' column
df['title'] = df['title'].apply(preprocess_text)

df.to_csv("sentiment_analysis_results.csv", index=True)

print(df["title"])

0       Presiden Joko Widodo menggelar rapat dengan se...
1       Halo SobATRBPN simak bersama yuk Kegiatan Waki...
2       Reposted from atrbpn  Halo SobATRBPN Kegiatan ...
3       Halo SobATRBPN simak bersama yuk Kegiatan Waki...
4       repost atrbpn  Halo SobATRBPN simak bersama yu...
                              ...                        
1222    Menteri Investasi Bahlil Lahadalia mengatakan ...
1223    Minimnya minat investor swasta dalam proses pe...
1224    RT kompascom Kepala Otorita Ibu Kota Nusantara...
1225    Kepala Otorita Ibu Kota Nusantara OIKN Bambang...
1226    Kemenkeu menggelontorkan sederet insentif perp...
Name: title, Length: 1227, dtype: object


In [7]:
# Step 2: Load model dan tokenizer

# model_name = "sarahlintang/IndoBERT"
# tokenizer = AutoTokenizer.from_pretrained("sarahlintang/IndoBERT")
# model = AutoModel.from_pretrained(model_name)

model_name = "indolem/indobert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModel.from_pretrained(model_name)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
import os

# Buat direktori jika belum ada
save_directory = "C:/Users/labib/Documents/Kuliah/Materi/Sem 6/MPI/Artikel ilmiah/model"
# save_directory = "E:/DataAbie/Kuliah\MPI/Artikel Ilmiah/model_saved"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Simpan model
torch.save(model.state_dict(), os.path.join(save_directory, "sentiment_model.pth"))

In [ ]:
# Menyimpan model classifier
# torch.save(model.state_dict(), "models/sentiment_model.pth")

In [22]:
# Step 3: Preprocess teks
def preprocess_text(text):
    encoded_input = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    return encoded_input

In [23]:
# Step 4: Definisikan model sentimen
class SentimentClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(SentimentClassifier, self).__init__()
        self.bert_model = bert_model
        self.rcnn = nn.GRU(bert_model.config.hidden_size, bert_model.config.hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(bert_model.config.hidden_size * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state
        _, hidden = self.rcnn(sequence_output)
        hidden = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(hidden)
        logits = self.fc(hidden)
        return logits

In [24]:
# Step 5: Load label encoder
label_encoder = LabelEncoder()
label_encoder.classes_ = ['negative', 'neutral', 'positive']  # Ganti dengan kelas sentimen yang sesuai

In [49]:
# Step 6: Load model sentimen yang telah dilatih
num_classes = len(label_encoder.classes_)
model_sentiment = SentimentClassifier(model, num_classes)
dir = os.listdir(save_directory)
model_path = ""
for file in dir:
    model_path = file
print(model_path)
model_sentiment.load_state_dict(torch.load("C:/Users/labib/Documents/Kuliah/Materi/Sem 6/MPI/Artikel ilmiah/model/sentiment_model.pth"))  # Ganti dengan path model yang telah dilatih
model_sentiment.eval()

sentiment_model.pth


RuntimeError: Error(s) in loading state_dict for SentimentClassifier:
	Missing key(s) in state_dict: "bert_model.embeddings.position_ids", "bert_model.embeddings.word_embeddings.weight", "bert_model.embeddings.position_embeddings.weight", "bert_model.embeddings.token_type_embeddings.weight", "bert_model.embeddings.LayerNorm.weight", "bert_model.embeddings.LayerNorm.bias", "bert_model.encoder.layer.0.attention.self.query.weight", "bert_model.encoder.layer.0.attention.self.query.bias", "bert_model.encoder.layer.0.attention.self.key.weight", "bert_model.encoder.layer.0.attention.self.key.bias", "bert_model.encoder.layer.0.attention.self.value.weight", "bert_model.encoder.layer.0.attention.self.value.bias", "bert_model.encoder.layer.0.attention.output.dense.weight", "bert_model.encoder.layer.0.attention.output.dense.bias", "bert_model.encoder.layer.0.attention.output.LayerNorm.weight", "bert_model.encoder.layer.0.attention.output.LayerNorm.bias", "bert_model.encoder.layer.0.intermediate.de

In [35]:
# Step 7: Definisikan fungsi untuk melakukan analisis sentimen
def analyze_sentiment(text):
    encoded_input = preprocess_text(text)
    with torch.no_grad():
        logits = model_sentiment(encoded_input['input_ids'], encoded_input['attention_mask'])
    probabilities = torch.softmax(logits, dim=1).tolist()[0]
    sentiment_label = label_encoder.inverse_transform([probabilities.index(max(probabilities))])
    sentiment_score = max(probabilities)
    return sentiment_label[0], sentiment_score

In [36]:
# Step 8: Analisis sentimen pada setiap tweet dalam dataset
sentiments = []
sentiment_scores = []

for tweet in tweets:
    sentiment_label, sentiment_score = analyze_sentiment(tweet)
    sentiments.append(sentiment_label)
    sentiment_scores.append(sentiment_score)

# Tambahkan kolom sentimen dan skor sentimen ke dalam dataframe
df["sentiment"] = sentiments
df["sentiment_score"] = sentiment_scores

# Save the updated dataframe to a new CSV file
# output_path = "sentiment_analysis_results.csv"
df.to_csv(output_path, index=False)

# Cetak dataframe dengan kolom sentimen dan skor sentimen
print(df[["title", "sentiment", "sentiment_score"]])

TypeError: only integer scalar arrays can be converted to a scalar index